# <span style="color:gray">ipyrad-analysis toolkit:</span> vcf_to_hdf5

Many genome assembly tools store SNP data in VCF format (variant call format), a tabular data format representing variant calls relative to a reference genome with associated metadata. To make analyses run a bit faster in ipyrad we instead use a database format called HDF5 which allows us to pull out and apply filtering on SNPs much more quickly. To make it easier to work with our preferred format (HDF5) we provide a tool (`ipa.vcf_to_hdf5()`) to convert VCF files to HDF5. This tool makes it possible to analyse any genomic data in the ipyrad.analysis toolkit, even if it was not assembled in ipyrad.

### When and how to use this tool:

The `vcf_to_hdf5` tool requires you to enter an `ld_block_size` argument which is used to assign SNPs to separate linkage blocks. SNPs that are located on the same linkage block are assumed to be in linkage disequilibrium (i.e., they are statistically correlated), which can introduce biases to many SNP based inference programs (e.g., PCA, STRUCTURE). This tool allows you to set a distance in bp at which linkage is expected to have decayed. This can be measured or estimated. For distantly related taxa it is likely a very short distance, whereas for closely related samples it may be much longer. 

If your data were assembled in ipyrad then the HDF5 output treats every RAD locus as a distinct linkage block, meaning that the `ld_block_size` is approximately 100-300bp, depending on the length of your loci. Alternatively, if you load in VCF data from an external tool then each chromosome/scaffold is by default treated as a linkage block, which is much too large. By setting the `ld_block_size` you can convert VCF to a new HDF5 for either of these two datasets, and create datasets with similar levels of linkage.


#### Loading a VCF from external software
The place where this tool is most generally useful is for loading in VCF data assembled from other tools, such as STACKS or GATK. If you set `ld_block_size` to 10000 then every 10000bp window will be assigned a unique linkage block in the HDF5 file. As such, a 5Mb chromosome will contain 500 linkage blocks. Thus, when you run a PCA analysis in `ipa.pca` it will randomly sample 1 unlinked SNP per linkage block to return a set of 500 unlinked SNPs from this chromosome. See the examples below.


#### Modifying linkage on an existing ipyrad assembly
If your data are RAD-seq loci assembled in ipyrad then each RAD locus will already be assigned to a separate linkage block in the HDF5 output. If your data is a *de novo* assembly then there is no reason to use this tool, but if your data is from a *reference*-based assembly then you can optionally modify the linkage blocks of your data using this tool. For example, if you want to combine all RAD loci that are within 50Kb of each other onto the same linkage block then you can enter `ld_block_size=50000`. See the example below.

In [17]:
import h5py
import numpy as np

# load arrays from hdf5; could be more efficient...
with h5py.File("/tmp/heliconius.snps.hdf5", 'r') as io5:
    snps = io5["snps"][:, 0:1000]
    print(snps.shape)
    
    genos = io5["genos"][0:1000, :, :]
    print(genos.shape)

(92, 1000)
(1000, 92, 2)


In [20]:
diplo = genos.sum(axis=2).T
diplo[diplo == 18] = 9
cov = np.sum(diplo[:, :] != 9, axis=0)
cov[:20]

array([90, 90, 90, 90, 92, 92, 91, 91, 91, 91, 90, 91, 92, 92, 92, 92, 92,
       92, 92, 92])

In [115]:
mg = np.ma.array(data=genos, mask=genos==9)
nhaplos = (~mg.mask).sum(axis=2).sum(axis=1)
nhaplos.shape

(1000,)

In [117]:
# subselect 10 samples (sidx)
mg = np.ma.array(data=genos, mask=genos==9)
sub = mg[:, 10:20]
sub.shape

(1000, 10, 2)

In [127]:
common = sub.sum(axis=2).mean(axis=1).round().astype(int).data
common.shape, common[:10]

((1000,), array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0]))

In [126]:
diplos = sub.sum(axis=2).data.T
diplos.shape, diplos[:10]

((10, 1000),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint64))

In [147]:
x0 = (mg == 0).sum(axis=2).sum(axis=1).data
x1 = (mg == 1).sum(axis=2).sum(axis=1).data
x1 / (x0 + x1)

array([0.16111111, 0.02222222, 0.01111111, 0.06111111, 0.05978261,
       0.01086957, 0.01098901, 0.01098901, 0.01098901, 0.02747253,
       0.01111111, 0.04395604, 0.01086957, 0.02717391, 0.09782609,
       0.4076087 , 0.01086957, 0.95652174, 0.02173913, 0.08152174,
       0.0326087 , 0.09782609, 0.09782609, 0.05434783, 0.02717391,
       0.09782609, 0.04347826, 0.01086957, 0.32065217, 0.07608696,
       0.07608696, 0.29347826, 0.07608696, 0.13043478, 0.01086957,
       0.01086957, 0.01086957, 0.04347826, 0.77717391, 0.80978261,
       0.01086957, 0.4076087 , 0.03804348, 0.01086957, 0.21195652,
       0.04347826, 0.0326087 , 0.01086957, 0.01098901, 0.10869565,
       0.02173913, 0.01630435, 0.10869565, 0.01086957, 0.06521739,
       0.0326087 , 0.07608696, 0.07608696, 0.01086957, 0.01098901,
       0.01111111, 0.01111111, 0.11111111, 0.01648352, 0.11538462,
       0.01086957, 0.02173913, 0.01086957, 0.01086957, 0.09782609,
       0.02717391, 0.01630435, 0.02173913, 0.0326087 , 0.10869

In [164]:
max_ages = [10, 20, 30]
min_ages = [5, 10, 20]

astring = f"""
    This is a string where I want to list some variables, 
    for example, list {min_ages} here, 
    and {max_ages} here. 
    Finally, skip this junk here which is part of the R 
    code:
    for (i in xyz) {{
        x <- ...
    }}
    """

print(astring)


    This is a string where I want to list some variables, 
    for example, list [5, 10, 20] here, 
    and [10, 20, 30] here. 
    Finally, skip this junk here which is part of the R 
    code:
    for (i in xyz) {
        x <- ...
    }
    


In [22]:
marr = np.ma.array(
    data=diplo[:, :], 
    mask=diplo[:, :] == 9,
)





masked_array(
  data=[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [--, --, --, ..., 0, 0, 2],
        [0, 0, 0, ..., 0, 0, 2]],
  mask=[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [False, False, False, ..., False, False, False]],
  fill_value=999999,
  dtype=uint64)

### Required software

In [1]:
# conda install ipyrad -c conda-forge -c bioconda

In [5]:
import itertools
import ipyrad.analysis as ipa

### Example 1: Download a  RAD-seq VCF file assembled from STACKS
To demonstrate the `vcf2hdf5` tool we will use a VCF file from an empirical dataset that was assembled from an external software tool (STACKS). The block below uses the `ipa.download` function to download the file from this study on DRYAD: (https://datadryad.org/stash/dataset/doi:10.5061/dryad.1rn8pk0pz). 

In [3]:
# the URL of a publicy available stacks-assembled dataset
DRYAD_VCF_URL = "https://datadryad.org/stash/downloads/file_stream/279287"

In [4]:
# download file to the requested location and decompress
ipa.download(url=DRYAD_VCF_URL, path="/tmp/wildlabdanio.vcf.gz", gunzip=True);

file already exists: /tmp/wildlabdanio.vcf.gz
decompressed file already exists: /tmp/wildlabdanio.vcf


### Example 1: Convert STACKS VCF to HDF5
In this example the dataset includes 240K SNPs across 223 scaffolds. We apply the `scaffolds` argument to select only the first 12 scaffolds (these are not necessarily the largest ones, so you generally need to look at the VCF to find scaffold (CHROM) names/numbers). We also apply the `ld_block_size` option to break up the assignment of SNPs on the scaffold to instead group into "linkage blocks". The `chunksize` argument only affects the speed and memory usage; larger values will make the tool run faster, but too large can lead to memory errors. 

In [5]:
# init a converter tool instance
tool = ipa.vcf_to_hdf5(
    data="/tmp/wildlabdanio.vcf.gz",
    name="wildlabdanio",
    workdir="/tmp",
    ld_block_size=20000,
    chunksize=2500,
    scaffolds=range(1, 13),  # only select CHROMS 1-12
)
tool.run(force=True)

Indexing VCF to HDF5 database file
VCF: 122351 SNPs; 12 scaffolds
[####################] 100% 0:02:17 | converting VCF to HDF5 
HDF5: 122351 SNPs; 8868 linkage groups
SNP database written to /tmp/wildlabdanio.snps.hdf5


### Example 1: quick analysis of SNP data
The PCA analysis is an example of an ipyrad-analysis tool that uses the linkage information to enable sampling of unlinked SNPs. Here we run 20 replicate analyses where each one randomly samples a SNP from every linkage block containing a SNP. The results represent variation over the sampled sets. 

In [6]:
# init a PCA tool and run 
tool = ipa.pca(
    data="/tmp/wildlabdanio.snps.hdf5",
    mincov=0.9, 
    minmaf=0.025, 
    impute_method="sample",
)

# run inference
tool.run(nreplicates=20)

AttributeError: 'SNPsExtracter' object has no attribute 'get_masks_chunk'

In [6]:
# create a dictionary by grouping samples based on the "_" prefix in sample names 
IMAP = {
    i: list(j) for (i, j) in 
    itertools.groupby(tool.names, key=lambda x: x.split("_")[0])
}

In [7]:
# draw with group assignment colors (the sample UT_30 may be contaminant?)
#tool.draw(0, 1, imap=IMAP);

### Example 2: Download a whole genome shotgun VCF
Download the VCF file from DRYAD for study using whole genome sequencing to call SNPs among ~100 Heliconius butterflies: (https://datadryad.org/stash/dataset/doi:10.5061/dryad.sk2pd88). If you take a look at the VCF file you can see that it has already been "cleaned", in the sense that it contains only the genotype calls and not a huge addition of metadata. This allows us to read it in in larger chunks (chunksize) without worrying about memory issues.

In [8]:
DRYAD_VCF_WGS_URL = "https://datadryad.org/stash/downloads/file_stream/512322"
ipa.download(url=DRYAD_VCF_WGS_URL, path="/tmp/heliconius.vcf.gz", force=True, gunzip=True);

file already exists: /tmp/heliconius.vcf.gz
decompressed file already exists: /tmp/heliconius.vcf


In [9]:
# run to convert the gzipped VCF to .snps.hdf5 format.
tool = ipa.vcf_to_hdf5(
    data="/tmp/heliconius.vcf.gz",
    name="heliconius",
    workdir="/tmp",
    ld_block_size=10000,
    chunksize=10000,
    scaffolds=["chr1", "chr2", "chr3", "chr4", "chr5", "chr6"],
)
tool.run(force=True)

Indexing VCF to HDF5 database file
VCF: 3717863 SNPs; 6 scaffolds
[####################] 100% 0:06:12 | converting VCF to HDF5 
HDF5: 3717863 SNPs; 6984 linkage groups
SNP database written to /tmp/heliconius.snps.hdf5


In [ ]:
# create a dictionary by grouping samples based on the "." prefix in sample names 
IMAP = {
    i: list(j) for (i, j) in 
    itertools.groupby(tool.names, key=lambda x: x.split(".")[0])
}

In [ ]:
# init a PCA tool and run 
tool = ipa.pca(
    data="/tmp/heliconius.snps.hdf5",
    mincov=0.9, 
    minmaf=0.1, 
    impute_method="sample",
)
tool.run(nreplicates=10)

Samples: 92
Sites before filtering: 3717863
Filtered (indels): 0


In [42]:
# run to convert the gzipped VCF to .snps.hdf5 format.
tool = ipa.vcf_to_hdf5(
    data="/tmp/heliconius.vcf.gz",
    name="heliconius",
    workdir="/tmp",
    ld_block_size=10000,
    chunksize=2500,
    scaffolds=["chr1", "chr2", "chr3", "chr4", "chr5", "chr6"],
)
tool.run(force=True)

Indexing VCF to HDF5 database file
VCF: 14406386 SNPs; 21 scaffolds
[####################] 100% 0:22:03 | converting VCF to HDF5 
HDF5: 14406386 SNPs; 1270 linkage group
SNP database written to /tmp/heliconius.snps.hdf5


In [ ]:
# init a PCA tool and run 
tool = ipa.pca(
    data="/tmp/heliconius.snps.hdf5",
    mincov=0.9, 
    minmaf=0.1, 
    impute_method="sample",
)
tool.run(nreplicates=10)

Samples: 92
Sites before filtering: 3717863
Filtered (indels): 0


In [43]:
# get grouping based on prefix of sample names
IMAP = {}
for name in tool.names:
    prefix = name.split(".")[0]
    if prefix in IMAP:
        IMAP[prefix].append(name)
    else:
        IMAP[prefix] = [name]

If you are converting a VCF file assembled from some other tool (e.g., GATK, freebayes, etc.) then you will need to install the `htslib` and `bcftools` software and use them as described below. 

 When working with WGS data you will also need the two additional software tools below for filtering, which you can install with conda. 

In [ ]:
# conda install htslib -c conda-forge -c bioconda
# conda install bcftools -c conda-forge -c bioconda

#### Pre-filter WGS data for conversion
You can use the program `bcftools` to pre-filter your data to exclude indels and low quality SNPs. If you ran the `conda install` commands above then you will have all of the required tools installed. To achieve the format that ipyrad expects you will need to exclude indel containing SNPs (this may change in the future). Further quality filtering is optional. The example below reduced the size of a VCF data file from ~5Gb to 80Mb! VCF contains a lot of information that you do not need to retain through all of your analyses. We will keep only the final genotype calls.  

Note that the code below is a bash script. You can run this from a terminal, or in a jupyter notebook by appending the (%%bash) header like below. 

In [38]:
%%bash

# compress the VCF file if not already done (creates .vcf.gz)
bgzip /tmp/heliconius.vcf

# tabix index the compressed VCF (creates .vcf.gz.tbi)
tabix -f /tmp/heliconius.vcf.gz

# remove multi-allelic SNPs and INDELs and PIPE to next command
bcftools view -m2 -M2 -i'CIGAR="1X" & QUAL>30' /tmp/heliconius.vcf.gz -Ou | 

    # remove extra annotations/formatting info and save to new .vcf
    bcftools annotate -x FORMAT,INFO  > /tmp/heliconius.cleaned.vcf.gz
    
# recompress the final file (create .vcf.gz)
bgzip /tmp/heliconius.cleaned.vcf.gz

[filter.c:2914 filters_init1] Error: the tag "CIGAR" is not defined in the VCF header
Failed to read from standard input: unknown file type


### A peek at the cleaned VCF file

In [3]:
# load the VCF as an datafram
dfchunks = pd.read_csv(
    "/home/deren/Documents/ipyrad/sandbox/Macaque-Chr1.clean.vcf.gz",
    sep="\t", 
    skiprows=1000, 
    chunksize=1000,
)

# show first few rows of first dataframe chunk
next(dfchunks).head()

NC_018152.2  51273  .  G  A    280.482 ..1 ..2  GT  0/0  ... 0/0.9 0/0.10  \
0  NC_018152.2  51292  .  A  G  16750.300   .   .  GT  1/1  ...   1/1      .   
1  NC_018152.2  51349  .  A  G    628.563   .   .  GT  0/0  ...   0/0    0/0   
2  NC_018152.2  51351  .  C  T    943.353   .   .  GT  0/0  ...   0/0    0/0   
3  NC_018152.2  51352  .  G  A    607.681   .   .  GT  0/0  ...   0/0    0/0   
4  NC_018152.2  51398  .  C  T    510.120   .   .  GT  0/0  ...   0/0    0/0   

  0/0.11 0/0.12 0/0.13 0/0.14 0/0.15 0/0.16 0/0.17 0/1.1  
0    1/1    1/1    1/1    1/1    0/0    1/1    1/1   1/1  
1    0/0    0/0    0/0    0/0    0/0    0/0    0/0   0/0  
2    0/0    0/0    0/0    0/0    0/0    0/0    0/0   0/0  
3    0/0    0/0    0/0    0/0    0/0    0/0    0/0   0/0  
4    0/0    0/0    0/0    0/0    0/0    0/0    0/0   0/0  

[5 rows x 29 columns]

### Converting clean VCF to HDF5 
Here I using a VCF file from whole genome data for 20 monkey's from an unpublished study (in progress). It contains >6M SNPs all from chromosome 1. Because many SNPs are close together and thus tightly linked we will likely wish to take linkage into account in our downstream analyses.

The ipyrad analysis tools can do this by encoding linkage block information into the HDF5 file. Here we encode `ld_block_size` of 20K bp. This breaks the 1 scaffold (chromosome) into about 10K linkage blocks. See the example below of this information being used in an ipyrad PCA analysis. 

In [4]:
# init a conversion tool
converter = ipa.vcf_to_hdf5(
    name="Macaque_LD20K",
    data="/home/deren/Documents/ipyrad/sandbox/Macaque-Chr1.clean.vcf.gz",
    ld_block_size=20000,
)

# run the converter
converter.run()

Indexing VCF to HDF5 database file
VCF: 6094152 SNPs; 1 scaffolds
[####################] 100% 0:02:22 | converting VCF to HDF5 
HDF5: 6094152 SNPs; 10845 linkage group
SNP database written to ./analysis-vcf2hdf5/Macaque_LD20K.snps.hdf5


### Downstream analyses
The data file now contains 6M SNPs across 20 samples and N linkage blocks. By default the PCA tool subsamples a single SNP per linkage block. To explore variation over multiple random subsamplings we can use the `nreplicates` argument. 

In [5]:
# init a PCA tool and filter to allow no missing data
pca = ipa.pca(
    data="./analysis-vcf2hdf5/Macaque_LD20K.snps.hdf5",
    mincov=1.0, 
)

Samples: 20
Sites before filtering: 6094152
Filtered (indels): 0
Filtered (bi-allel): 0
Filtered (mincov): 794597
Filtered (minmap): 0
Filtered (combined): 794597
Sites after filtering: 5299555
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


### Run a single PCA analysis from subsampled unlinked SNPs

In [6]:
pca.run_and_plot_2D(0, 1, seed=123);

Subsampling SNPs: 10841/5299555


<svg class="toyplot-canvas-Canvas" height="300.0px" id="tbb2b3ed2710a45baa108ba1e1573b3cc" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> SRR2981140 SRR2981114 nemestrina2 SRR4454020 SRR5947292 SRR5947293 SRR7588781 SRR5947294 SRR2981139 sylvanus fasno SRR4453966 SRR4454026 silenus fuscata2 fasso SRR8285768 DRR002233 SRR5628058 SRR1024051 -25 0 25 50 75 PC0 (25.4%) explained -25 0 25 PC1 (14.4%) explained

### Run multiple PCAs over replicates of subsampled SNPs 
Here you can see the results for a *different* 10K SNPs that are sampled in each replicate iteration. If the signal in the data is robust then we should expect to see the points clustering at a similar place across replicates. Internally ipyrad will rotate axes to ensure the replicate plots align despite axes swapping (which is arbitrary in PCA space). You can see this provides a better view of uncertainty in our estimates than the plot above (and it looks cool!)

In [7]:
pca.run_and_plot_2D(0, 1, seed=123, nreplicates=25);

Subsampling SNPs: 10841/5299555


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t1ab6529cdf2c4192aa20c8b1d221f741" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> SRR2981140 SRR2981114 nemestrina2 SRR4454020 SRR5947292 SRR5947293 SRR7588781 SRR5947294 SRR2981139 sylvanus fasno SRR4453966 SRR4454026 silenus fuscata2 fasso SRR8285768 DRR002233 SRR5628058 SRR1024051 -25 0 25 50 75 PC0 (25.4%) explained -25 0 25 PC1 (14.4%) explained

More details on running PCAs, toggling options, and styling plots can be found in our [ipyrad.analysis PCA tutorial](https://ipyrad.readthedocs.io/en/latest/API-analysis/cookbook-pca.html)